In [136]:
import numpy as np
import typing
class CNF(object):
    def __init__(self, clauses: np.ndarray):
        """
        Clauses is a two-dimensional array with the following semantics for
        `val = clauses[i, j]`:
            * If val == 0, that space is empty.
            * If val > 0, then the 1-indexed variable val occurs in clause i, unnegated.
            * if val < 0, then the 1-indexed variable val occurs in clause i, negated.
        We assume that clauses[i] consists of some number of nonzero elements,
        followed by zero.
        """
        self.clauses = clauses
        self.num_variables = np.abs(clauses).max()
        self.variables = set(np.abs(self.clauses).flatten()) - {0}

        # Start off assuming every disjunct of every clause is True.
        # If any row of one of the sheets of this array is ever all Falses,
        # then there is a contradiction. The 0th sheet records our current
        # view of known assignments, and subsequent sheets allow scratch work
        # in our backtracking.
        self.positive = np.zeros((self.num_variables, ) + self.clauses.shape,
                                 dtype=bool)
        self.positive[0] = (self.clauses != 0)
        
        # Start off assuming every disjunct of every clause is False.
        # If each row of this array contains a nonzero entry, then
        # we have found a satisfying assignment.
        self.negative = np.zeros((self.num_variables, ) + self.clauses.shape,
                                 dtype=bool)
        
        # Keys are tuples of assignments, sorted by absolute value.
        # For example (-1, 2, 3, -6). `values` a 3-dimensional array
        # where `masks[assignment][0]` is the mask to do bitwise AND
        # with self.positive to get the value after doing the updates
        # in `assignments`. 
        
        self.assignments = set()
        self.unassigned = self.variables - self.assignments
        
        
        self.masks = {}
        # compute masks for all assignments of a single variable.
        to_assign = []
        for variable in range(1, self.num_variables + 1):
            for assignment in [-1, 1]:
                mask = np.ndarray((2, ) + self.clauses.shape, dtype=bool)
                positive_mask = self.positive[0] & (self.clauses != -assignment * variable)
                mask[0] = positive_mask
                negative_mask = (self.clauses == assignment * variable)
                mask[1] = negative_mask
                self.masks[variable * assignment] = mask
                if (~positive_mask.any(axis=1)).any():
                    to_assign.append(-assignment * variable)
        self.assign(*to_assign)

    def solve_naive(self):
        """
        Try all possible combinations in a depth-first search.
        
        Returns a satisfying solution if one exists, otherwise returns None.
        """
        possible_assignments = [
            [variable, -variable]
            for variable in sorted(self.variables)
            if not (variable in self.assignments or -variable in self.assignments)
        ]
        def visit(stack):
            pass
        def depth_first(possibilities, start, stack, depth=0):
            for i in range(start, len(possibilities)):
                for possibility in possibilities[i]:
                    stack.add(possibility)
                    print(f'stack={stack}')
                    mask = self.masks[possibility]
                    self.positive[depth+1] = self.positive[depth] & mask[0]
                    if (~self.positive[depth+1].any(axis=1)).any():
                        print(f'contradiction: {stack}')
                        # Contradiction reached.
                        return (False, None)
                    self.negative[depth+1] = self.negative[depth] | mask[1]
                    if self.negative[depth+1].any(axis=1).all():
                        # found a satisfying assignment:
                        return (True, list(stack))
                    solution = None
                    if i+1 < len(possibilities):
                        found, solution = depth_first(possibilities, i+1, stack, depth=depth+1)
                        if found:
                            return (True, solution)
                    else:
                        print(f'{i}, {len(possibilities)}, stack')
                    stack.remove(possibility)
            assert False, 'Unreachable?'
        return depth_first(possible_assignments, 0, set(), depth=0)

    def is_satisified(self, depth=0):
        return self.negative[depth].any(axis=1).all()

    def is_inconsistent(self, depth=0):
        return (~self.positive[depth].any(axis=1)).any()

    @classmethod
    def load_dimacs(cls, s: str):
        """Loads a boolean expression from a string.

        Based off sympy.logic.utilities.dimacs
        https://github.com/sympy/sympy/blob/57fcd5a941d7c47106bd63fd7b3d79ac032b636b/sympy/logic/utilities/dimacs.py
        """
        import re
        clauses = []
        pComment = re.compile(r'c.*')
        pStats = re.compile(r'p\s*cnf\s*(\d*)\s*(\d*)')
        lines = [line.strip() for line in s.splitlines()
                 if not pComment.match(line) and not pStats.match(line)]
        clauses = []
        for line in lines:
            nums = [int(num) for num in line.rstrip('\n').split(' ')
                    if num not in {'', '0'}]
            if nums:
                clauses.append(nums)
        a = np.zeros((len(clauses), max(map(len, clauses))))
        import itertools
        return cls(np.array(list(itertools.zip_longest(*clauses,fillvalue=0)), dtype=np.int16).T)

    def evaluate(self) -> np.ndarray:
        """
        Returns a new clause array with self.assignments applied.
        """
        # Remove satisified clauses:
        unsatisfied_rows = (~self.negative[0].any(axis=1)).nonzero()
        new_clauses = self.clauses[unsatisfied_rows]

        # Remove contradictory atoms:
        for assignment in self.assignments:
            new_clauses *= (new_clauses != -assignment)

        # Shrink the array to remove zero columns:
        new_clauses = new_clauses[np.unravel_index(np.argsort(-np.abs(new_clauses), axis=1), new_clauses.shape)]
        t = new_clauses.T
        new_clauses = t[~(t==0).all(1)].T
        return new_clauses

    def assign(self, *assignments: typing.List[int]):
        assignments = set(assignments)
        todo = assignments.copy()

        while todo:
            assignment = todo.pop()
            self.assignments.add(assignment)
            if -assignment in self.masks:
                del self.masks[-assignment]
            if assignment not in self.masks:
                continue
            assignment_mask = self.masks.pop(assignment)
            self.positive[0] &= assignment_mask[0]
            self.negative[0] |= assignment_mask[1]
            for t, mask in list(self.masks.items()):
                mask[0] &= assignment_mask[0]
                mask[1] |= assignment_mask[1]
                if (~(mask[0].any(axis=1))).any():
                    # This assignment has become inconsistent, so remove it.
                    del self.masks[t]
                    if -t in self.assignments:
                        continue
                    elif -t in assignments:
                        continue
                    else:
                        print(f'Deduced %s from %s' % (-t, assignment))
                        assignments.add(-t)
                        todo.add(-t)

In [137]:
factor_35_dimacs = '''\
p cnf 6 6 
c Factors encoded in variables 1-3 and 4-6
c Target number: 35
-2 -5 0
1 0
3 0
4 0
6 0
2 5 0
'''
problem = CNF.load_dimacs(factor_35_dimacs)
print(problem.clauses)
list(map(frozenset, -problem.clauses))
# print(problem.assignments)
# problem.assign(-2)
# print(problem.assignments)
problem.solve_naive()

[[-2 -5]
 [ 1  0]
 [ 3  0]
 [ 4  0]
 [ 6  0]
 [ 2  5]]
stack={2}
stack={2, 5}
contradiction: {2, 5}
stack={5, -2}
stack={5, -2}


(True, [5, -2])

In [139]:
factor_143='''\
p cnf 8 12 
c Factors encoded in variables 1-4 and 5-8
c Target number: 143
c Factors: 13 x 11
3 2 0
2 6 0
1 0
-3 -2 0
8 0
-2 -7 -6 0
4 0
-3 -2 -6 0
5 0
-2 -6 0
-3 -7 0
3 7 0'''
problem = CNF.load_dimacs(factor_143)
print(problem.clauses, problem.assignments)
problem.solve_naive()


[[ 3  2  0]
 [ 2  6  0]
 [ 1  0  0]
 [-3 -2  0]
 [ 8  0  0]
 [-2 -7 -6]
 [ 4  0  0]
 [-3 -2 -6]
 [ 5  0  0]
 [-2 -6  0]
 [-3 -7  0]
 [ 3  7  0]] {8, 1, 4, 5}
stack={2}
stack={2, 3}
contradiction: {2, 3}
stack={3, -2}
stack={3, -2}
stack={3, -2, 6}
stack={3, 6, 7, -2}
contradiction: {3, 6, 7, -2}
stack={3, 7, -6, -2}
contradiction: {3, 7, -6, -2}
stack={7, -6, -3, -2}
contradiction: {7, -6, -3, -2}
stack={3, 7, -6, -3}
stack={3, 6, 7, -6, -3}
stack={3, 6, 7, -6, -3}
contradiction: {3, 6, 7, -6, -3}
stack={3, 7, -6, -3}
stack={3, 7, -6, -3}
contradiction: {3, 7, -6, -3}
stack={3, 7, -3}
contradiction: {3, 7, -3}
stack={7, -3}
stack={6, 7, -3}
stack={6, 7, -3}
3, 4, stack
stack={6, -7, -3}
contradiction: {6, -7, -3}
stack={-7, -6, -3}
stack={7, -7, -6, -3}
3, 4, stack
stack={-7, -6, -3}
contradiction: {-7, -6, -3}
stack={7, -7, -3}
3, 4, stack
stack={-7, -3}
contradiction: {-7, -3}
stack={6, -7}
stack={6, 7, -7}
3, 4, stack
stack={6, -7}
3, 4, stack


AssertionError: Unreachable?

In [62]:
len(visited), len(set(visited))

(80, 80)